In [482]:
from tweepy import API
from tweepy import Cursor
#from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import RateLimitError
import pandas as pd
import re


In [152]:
consumer_token='2qhjD55Se9NGKRTBuZyb6Ki0Z'
consumer_secret= 'KooojZyWYxp7iu6gerfpwyJB1xbOVZHxRXYcO8L7VsF0CqQPAt'
access_token='352599159-PO9igRuIq6bWZgYcLU9AAeONtQarHyswa9SsW3hr'
access_token_secret='qhPSxUDrBIq8AQu4nTeiXBlkC5rthIlFcqGOYfOEvkGrX'




In [157]:
#setting up authorization
auth = OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
# Creation of the actual interface, using authentication
api = API(auth)
api.wait_on_rate_limit = True
 
# Sample method, used to update a status
#api.update_status('Hello Python Central!')

In [154]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [474]:
#setup
acceptable_locs = {
    'AL':['al','alabama'],
    'AK':['ak','alaska'], 
    'AZ':['az','arizona'], 
    'AR':['ar','arkansas'], 
    'CA':['ca','california'], 
    'CO':['co','colorado'], 
    'CT':['ct','connecticut'], 
    'DE':['de','delaware'], 
    'FL':['fl','florida'], 
    'GA':['ga','georgia'], 
    'HI':['hi','hawaii'], 
    'ID':['id','idaho'], 
    'IL':['il','illinois'], 
    'IN':['in','indiana'], 
    'IA':['ia','iowa'], 
    'KS':['ks','kansas'], 
    'KY':['ky','kentucky'], 
    'LA':['la','louisiana'], 
    'ME':['me','maine'], 
    'MD':['md','maryland'], 
    'MA':['ma','massachusetts'], 
    'MI':['mi','michigan'], 
    'MN':['mn','minnesota'], 
    'MS':['ms','mississippi'], 
    'MO':['mo','missouri'], 
    'MT':['mt','montana'],
    'NE':['ne','nebraska'], 
    'NV':['nv','nevada'], 
    'NH':['nh','newhampshire'], 
    'NJ':['nj','newjersey','jersey'], 
    'NM':['nm','newmexico'], 
    'NY':['ny','newyork','nyc'],
    'NC':['nc','northcarolina'],
    'ND':['nd','northdakota'],
    'OH':['oh','ohio'],
    'OK':['ok','oklahoma'],
    'OR':['or','oregon'],
    'PA':['pa','pennsylvania'],
    'RI':['ri','rhodeisland','rhode'],
    'SC':['sc','southcarolina'],
    'SD':['sd','southdakota'],
    'TN':['tn','tennessee'],
    'TX':['tx','texas'],
    'UT':['ut','utah'],
    'VT':['vt','vermont'],
    'VA':['va','virginia'],
    'WA':['wa','washington'],
    'WV':['wv','westvirginia'],
    'WI':['wi','wisconsin'],
    'WY':['wy','wyoming'],
}

#unpacking/melting above dictionary to list for direct access
acceptable_terms = []
for each in acceptable_locs.values():
    acceptable_terms.extend(each)

In [484]:
#funcs with api calls
def get_n_tweets_df(screen_name,n=10):
    wanted_status_keys = ['id','text','created_at','favorite_count','retweet_count','retweeter_ids','entities']
    temp = pd.DataFrame([tweet._json for tweet in Cursor(api.user_timeline, screen_name, include_rts=False).items(n)])
    temp['retweeter_ids'] = temp['id'].map(lambda x:api.retweeters(x))
    return temp[wanted_status_keys]
    
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except RateLimitError:
            time.sleep(15 *60)
    
def get_location_by_id(idnum):
    return api.get_user(idnum)._json['location']

def add_retweet_geos(df):
    df['retweeter_geos'] = df['retweeter_ids'].map(lambda x: [get_location_by_id(idnum) for idnum in x])
    return df




## caution, live api calls ahead!

In [ ]:
#api calls
booker_df = get_n_tweets_df('SenBooker')
booker_df['retweeter_locs'] = booker_df['retweeter_ids'].map(lambda x: [get_location_by_id(idnum) for idnum in x])

In [488]:
warren_df = get_n_tweets_df('SenWarren')
warren_df['retweeter_locs'] = warren_df['retweeter_ids'].map(lambda x: [get_location_by_id(idnum) for idnum in x])

## k we're done, relax


In [499]:
warren_df

,id,coordinates,text,created_at,favorite_count,retweet_count,rt_loc_counts
0,1095767417551241217,None,My plan to #EndCorruptionNow in 6 steps:\n✅Pad...,Wed Feb 13 19:31:25 +0000 2019,3681,1142,"{'MI': 1, 'OH': 2, 'ME': 1, 'IL': 3, 'PA': 1, ..."
1,1095420634291675139,None,You bet I do. https://t.co/d3mRz3d8lP,Tue Feb 12 20:33:25 +0000 2019,28958,3922,"{'FL': 2, 'OH': 1, 'PA': 2, 'NY': 3, 'NC': 1, ..."
2,1095374766792429568,None,.@realDonaldTrump wants 2 things from his Atto...,Tue Feb 12 17:31:10 +0000 2019,11301,3164,"{'CA': 10, 'NY': 6, 'WA': 1, 'IN': 5, 'NJ': 2,..."
3,1095096435652083712,None,Mary Hamilton successfully fought a court syst...,Mon Feb 11 23:05:10 +0000 2019,3768,983,"{'CA': 3, 'NC': 2, 'IN': 1, 'CO': 1, 'WA': 4, ..."
4,1095035939330637824,None,It’s too easy for the wealthy &amp; well-conne...,Mon Feb 11 19:04:47 +0000 2019,5588,1655,"{'FL': 4, 'KS': 1, 'KY': 1, 'CA': 4, 'NY': 2, ..."
5,1094008404933308417,None,A bank teller who takes a few 20s from the ban...,Fri Feb 08 23:01:44 +0000 2019,42313,12303,"{'NY': 5, 'CA': 7, 'OK': 1, 'NJ': 4, 'FL': 2, ..."
6,1093943381124894720,None,Ready. https://t.co/MI0OgJFZV9,Fri Feb 08 18:43:21 +0000 2019,19722,4213,"{'FL': 5, 'NC': 1, 'TX': 2, 'MI': 2, 'SC': 2, ..."
7,1093909943042871296,None,"Gigantic banks wrecked our economy, but the Fe...",Fri Feb 08 16:30:28 +0000 2019,2063,776,"{'NJ': 1, 'KY': 1, 'GA': 1, 'SC': 1, 'OH': 1, ..."
8,1093722297830121473,None,"My heart goes out to @RepDebDingell, and their...",Fri Feb 08 04:04:50 +0000 2019,2889,347,"{'GA': 2, 'OH': 4, 'IA': 2, 'WA': 2, 'NC': 2, ..."
9,1093608723686309895,None,Kathy Kraninger gutted the @CFPB rule that wou...,Thu Feb 07 20:33:32 +0000 2019,3772,1428,"{'KY': 1, 'GA': 3, 'OK': 1, 'OH': 1, 'WI': 1, ..."


In [485]:
#funcs with local scope

def state_name_collapser(string):
    """
    eliminates space after given keywords
    """
    string = re.sub(r'north[ ]','north',string)
    string = re.sub(r'south[ ]','south',string)
    string = re.sub(r'west[ ]','west',string)
    string = re.sub(r'new[ ]','new',string)
    return string

def get_real_location(loc):
    #normalizing input, seperating into list
    clean = state_name_collapser(loc.replace(',','').lower())
    clean_split = clean.split()
    
    #filtering for words that show up in our list of acceptable terms
    dropping_trash = [word for word in clean_split if word in acceptable_terms]
    
    #return key (ex 'NY' for 'new york') if associated value is found in dropping_trash
    output = ''
    for each,lst in acceptable_locs.items():
         for item in lst:
                if item in dropping_trash:
                    output += each
                    
    #HACK ALERT: if more than 1 associated key found (ex: 'california new york'-->'CANY', only returns 1 key (alphabetically)         
    return output[:2]

def locs_parser(lstoflocs):
    return list(filter(None,[get_real_location(name) for name in lstoflocs]))

def item_counter(lst):
    tempdic={}
    for each in lst:
        if each in tempdic:
            tempdic[each] += 1
        else:
            tempdic[each] = 1
    return tempdic


    

In [498]:

#local MUNGING
booker_df['parsed_locs'] = booker_df['retweeter_locs'].map(lambda x: locs_parser(x))
booker_df['rt_loc_counts']= booker_df['parsed_locs'].map(lambda x: item_counter(x))
warren_df['parsed_locs'] = warren_df['retweeter_locs'].map(lambda x: locs_parser(x))
warren_df['rt_loc_counts']= warren_df['parsed_locs'].map(lambda x: item_counter(x))

#drop unwanted SHITE
shite = ['place','coordinates','geo','retweeter_ids','entities','retweeter_locs','parsed_locs']

KeyError: 'retweeter_locs'

In [496]:
warren_df.drop(shite, axis=1, inplace=True)
booker_df.drop(shite, axis=1, inplace=True)


In [497]:
warren_df

,id,coordinates,text,created_at,favorite_count,retweet_count,rt_loc_counts
0,1095767417551241217,None,My plan to #EndCorruptionNow in 6 steps:\n✅Pad...,Wed Feb 13 19:31:25 +0000 2019,3681,1142,"{'MI': 1, 'OH': 2, 'ME': 1, 'IL': 3, 'PA': 1, ..."
1,1095420634291675139,None,You bet I do. https://t.co/d3mRz3d8lP,Tue Feb 12 20:33:25 +0000 2019,28958,3922,"{'FL': 2, 'OH': 1, 'PA': 2, 'NY': 3, 'NC': 1, ..."
2,1095374766792429568,None,.@realDonaldTrump wants 2 things from his Atto...,Tue Feb 12 17:31:10 +0000 2019,11301,3164,"{'CA': 10, 'NY': 6, 'WA': 1, 'IN': 5, 'NJ': 2,..."
3,1095096435652083712,None,Mary Hamilton successfully fought a court syst...,Mon Feb 11 23:05:10 +0000 2019,3768,983,"{'CA': 3, 'NC': 2, 'IN': 1, 'CO': 1, 'WA': 4, ..."
4,1095035939330637824,None,It’s too easy for the wealthy &amp; well-conne...,Mon Feb 11 19:04:47 +0000 2019,5588,1655,"{'FL': 4, 'KS': 1, 'KY': 1, 'CA': 4, 'NY': 2, ..."
5,1094008404933308417,None,A bank teller who takes a few 20s from the ban...,Fri Feb 08 23:01:44 +0000 2019,42313,12303,"{'NY': 5, 'CA': 7, 'OK': 1, 'NJ': 4, 'FL': 2, ..."
6,1093943381124894720,None,Ready. https://t.co/MI0OgJFZV9,Fri Feb 08 18:43:21 +0000 2019,19722,4213,"{'FL': 5, 'NC': 1, 'TX': 2, 'MI': 2, 'SC': 2, ..."
7,1093909943042871296,None,"Gigantic banks wrecked our economy, but the Fe...",Fri Feb 08 16:30:28 +0000 2019,2063,776,"{'NJ': 1, 'KY': 1, 'GA': 1, 'SC': 1, 'OH': 1, ..."
8,1093722297830121473,None,"My heart goes out to @RepDebDingell, and their...",Fri Feb 08 04:04:50 +0000 2019,2889,347,"{'GA': 2, 'OH': 4, 'IA': 2, 'WA': 2, 'NC': 2, ..."
9,1093608723686309895,None,Kathy Kraninger gutted the @CFPB rule that wou...,Thu Feb 07 20:33:32 +0000 2019,3772,1428,"{'KY': 1, 'GA': 3, 'OK': 1, 'OH': 1, 'WI': 1, ..."
